#### (1)  (No programming yet,) go to https://opensea.io/collection/boredapeyachtclub. and select all apes with “Solid gold” fur and sort them “Price high to low” .  Use the URL for the subsequent coding.

In [647]:
url1 = "https://opensea.io/collection/boredapeyachtclub?search[sortAscending]=false&search[stringTraits][0][name]=Fur&search[stringTraits][0][values][0]=Solid%20Gold"

#### (2)  Using Python or Java, write code that uses Selenium to access the URL from (1), click on each of the top-8 most expensive Bored Apes, and store the resulting details page to disk, “bayc_[N].htm” (replace [N] with the ape number).

In [700]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import requests
import re
from bs4 import BeautifulSoup
import pymongo
import http.client, urllib.parse
import json

In [649]:
driver = webdriver.Chrome(executable_path='/Users/prade/Downloads/chromedriver.exe')
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

top_n_xpath=[]

for i in range(1,9):
    top_n_xpath.append(f"/html/body/div[1]/div/main/div/div/div/div[5]/div/div[7]/div[3]/div[2]/div/div[{i}]")

for idx, i in enumerate(top_n_xpath):
    driver.get(url1)
    time.sleep(4) 
    top= driver.find_element(By.XPATH, value=i)
    top.click()
    time.sleep(3)
    with open(f"bayc_{idx+1}.htm", "w", encoding='utf-8') as f:
        f.write(driver.page_source)
        time.sleep(3)
        


driver.quit()

C:\Users\prade\AppData\Local\Temp\ipykernel_28808\1703432143.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/prade/Downloads/chromedriver.exe')


#### (3)  Write code that goes through all 8 htm files downloaded in (2) and stores each ape’s name (its number) and all its attributes in a document inside a MongoDB collection called “bayc”.

In [650]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["apes"]
mycol_ape = mydb["bayc"]

In [651]:
mycol_ape.collection_name

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'apes'), 'bayc.collection_name')

In [492]:
docn=[]

for i in range(1,9):
    with open(f"bayc_{i}.htm", "r", encoding='utf-8') as f:
        docn.append(f.read())
        

## soup and ape numbers        
soupn=[]
ape_num=[]

for idx, i in enumerate(docn):
    soupn.append(BeautifulSoup(i, 'html.parser'))
    ape_num.append(soupn[idx].find('h1').text)
    
## keys for attributes
ape_keys = []
for i in range(8):
    ape_keys.append(soupn[i].find_all('div', {'class':'Property--type'}))
    
## values for attributes
ape_values = []
for i in range(8):
    ape_values.append(soupn[i].find_all('div', {'class':'Property--value'}))
    
## rarity for attributes
ape_rarity = []
for i in range(8):
    ape_rarity.append(soupn[i].find_all('div', {'class':'Property--rarity'}))

            

ape_dic_all = []

for j in range(8):
    ape_dic={'ape_name_num':ape_num[j]}
    for i in range(len(ape_keys[j])):
        key=ape_keys[j][i].text
        value={'type':ape_values[j][i].text, 'rarity':ape_rarity[j][i].text}
        ape_dic[key]=value
    ape_dic_all.append(ape_dic)


#ape_dic_all

In [8]:
for i in range(8):
    mycol_ape.insert_one(ape_dic_all[i])

In [757]:
#mycol_ape.delete_many({})

In [1]:
#list(mycol_ape.find({}))

#### (4)  Yellow Pages uses GET requests for its search.  Using plain Python or Java (no Selenium), write a program that searches on yellowpages.com for the top 30 “Pizzeria” in San Francisco (no need to verify that the shop is actually selling pizzas, just search for “Pizzeria”, top 30 shops according to YP's "Default" sorting).  Save the search result page to disk, “sf_pizzeria_search_page.htm”.

In [652]:
url1= "https://www.yellowpages.com/search?search_terms=pizzeria&geo_location_terms=San+Francisco%2C+CA"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36 Edg/88.0.705.56' }

In [653]:
session1 = requests.session()
response1 = session1.get(url1, headers=headers)
response1

<Response [200]>

In [654]:
cookies1 = response1.cookies.get_dict()

In [655]:
# Parse the response
soup1 = BeautifulSoup(response1.text, 'html.parser')
#soup1

In [656]:
def saveString(html, filename="test.html"):
    try:
        file = open(filename,"w", encoding="utf-8")
        file.write(str(html))
        file.close()
    except Exception as ex:
        print('Error: ' + str(ex))


def loadString(f="test.html"):
    try:
        html = open(f, "r", encoding='utf-8').read()
        return(html)
    except Exception as ex:
        print('Error: ' + str(ex))

In [657]:
folder_to_Save = "C:/Users/prade/OneDrive/Documents/UC Davis/3. Winter/422 Data Design and Representation/HWs/Individual_Project_2/"

In [658]:
saveString(requests.get(url1, headers=headers).text, filename=folder_to_Save+"sf_pizzeria_search_page.htm")
time.sleep(5)

#### (5)  Using Python or Java, write code that opens the search result page saved in (4) and parses out all shop information (search rank, name, linked URL [this store’s YP URL], star rating If It Exists, number of reviews IIE, TripAdvisor rating IIE, number of TA reviews IIE, “$” signs IIE, years in business IIE, review IIE, and amenities IIE).  Please be sure to skip all “Ad” results.

In [659]:
products = []

page5=BeautifulSoup(loadString("sf_pizzeria_search_page.htm"), 'html.parser')

for idx,i in enumerate(page5.find_all('div' ,attrs={'class':'info'})):
    try:
        products.append({"sr_name": i.find('h2', attrs={'class':'n'}).text, \
                     "sr": re.search(r'^[^.]+',i.find('h2', attrs={'class':'n'}).text).group(0), \
                     "name": (i.find('h2', attrs={'class':'n'}).text).split('.', 1)[-1], \
                     "url_links": "https://www.yellowpages.com" + str(i.find('a')['href']), \
                     "star_rating": i.find('a', attrs={'class':'rating hasExtraRating'}).find('div', attrs={'class':'result-rating'})['class'][-1], \
                     "num_reviews": i.find('a', attrs={'class':'rating hasExtraRating'}).find('span', attrs={'class':'count'}).text, \
                     "ta_rating" : (i.find('div', attrs={'class':'ratings'})['data-tripadvisor'].split(","))[0], \
                     "ta_rating_num" : (i.find('div', attrs={'class':'ratings'})['data-tripadvisor'].split(","))[1], \
                     "dollar_signs" : i.find('div', attrs={'class':'price-range'}).text, \
                     "yrs_in_bussiness" : i.find('div', attrs={'class':'number'}).text, \
                     "review": i.find('p', attrs={'class':'body with-avatar'}).text, \
                     "amenities": i.find('div',attrs={'class':'amenities'}).text
                        })
    except:
        products.append({"sr_name": i.find('h2', attrs={'class':'n'}).text, \
                     "sr": re.search(r'^[^.]+',i.find('h2', attrs={'class':'n'}).text).group(0), \
                     "name": (i.find('h2', attrs={'class':'n'}).text).split('.', 1)[-1], \
                     "url_links": "https://www.yellowpages.com" + str(i.find('a')['href'])
                        })
    
    
    

#products

In [729]:
products = products[1:31]

In [730]:
len(products)

30

In [734]:
dict_222 = {'a':2}
dict_222

{'a': 2}

In [735]:
dict_222.update({'c':3, 'd':5})
dict_222

{'a': 2, 'c': 3, 'd': 5}

In [741]:
dict_222

{'a': 2, 'c': 3, 'd': 5}

In [742]:
dict_222.update({'c':3, 'd':5})
dict_222

{'a': 2, 'c': 3, 'd': 5}

#### (6)  Copy your code from (5).  Modify the code to create a MongoDB collection called “sf_pizzerias” that stores all the extracted shop information, one document for each shop.

In [709]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb_pizzas = client["pizzas"]
mycol_pizzas = mydb_pizzas["sf_pizzerias"]

In [710]:
mycol_pizzas.collection_name

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'pizzas'), 'sf_pizzerias.collection_name')

In [711]:
mycol_ape.delete_many({})

In [712]:
for i in (products):
    mycol_pizzas.insert_one(i)

In [714]:
#list(mycol_pizzas.find({}))

#### (7)  Write code that reads all URLs stored in “sf_pizzerias” and download each shop page.  Store the page to disk, “sf_pizzerias_[SR].htm” (replace [SR] with the search rank).

In [661]:
driver = webdriver.Chrome(executable_path='/Users/prade/Downloads/chromedriver.exe')
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

for i in mycol_pizzas.find():
    url=(i['url_links'])
    driver.get(url)
    time.sleep(4)
    with open(f"sf_pizzerias_"+i['sr']+".htm", "w", encoding='utf-8') as f:
        f.write(driver.page_source)
        time.sleep(3)

driver.quit()


C:\Users\prade\AppData\Local\Temp\ipykernel_28808\3404424686.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/prade/Downloads/chromedriver.exe')


WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: cannot determine loading status
from target frame detached
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x00C44AD3]
	(No symbol) [0x00BD9211]
	(No symbol) [0x00ACD890]
	(No symbol) [0x00ABF1E8]
	(No symbol) [0x00ABDC38]
	(No symbol) [0x00ABE617]
	(No symbol) [0x00AD7888]
	(No symbol) [0x00AD03A7]
	(No symbol) [0x00ACFEDF]
	(No symbol) [0x00AD0653]
	(No symbol) [0x00AC8DC5]
	(No symbol) [0x00AC89DF]
	(No symbol) [0x00AD4886]
	(No symbol) [0x00AD80F6]
	(No symbol) [0x00ABE9C1]
	(No symbol) [0x00AD4447]
	(No symbol) [0x00B36D07]
	(No symbol) [0x00B1F936]
	(No symbol) [0x00AF4788]
	(No symbol) [0x00AF5C1D]
	GetHandleVerifier [0x00EB9342+2502274]
	GetHandleVerifier [0x00EE7959+2692249]
	GetHandleVerifier [0x00EEABDC+2705180]
	GetHandleVerifier [0x00CF3480+643008]
	(No symbol) [0x00BE208F]
	(No symbol) [0x00BE7AB8]
	(No symbol) [0x00BE7BA5]
	(No symbol) [0x00BF273B]

#### (8)  Write code that reads the 30 shop pages saved in (7) and parses each shop’s address, phone number, and website.

In [739]:
textpages=[]
address=[]
phone=[]
website=[]


for i in range(1,31):
    try:
        loadString(f"sf_pizzerias_"+str(i)+".htm")
        page=BeautifulSoup(loadString(f"sf_pizzerias_"+str(i)+".htm"))
        phone.append(page.find('section',attrs={'id':'details-card'}).find('p', attrs={'class':'phone'}).text)
        website.append(page.find('section',attrs={'id':'details-card'}).find('p', attrs={'class':'website'}).text)
        add=str(page.find('section',attrs={'id':'details-card'}))
        idx1= add.find('Address: </span>')
        idx2=add.find('</p><p class')
        address.append(add[idx1+16:idx2])
        
    except:
        loadString(f"sf_pizzerias_"+str(i)+".htm")
        page=BeautifulSoup(loadString(f"sf_pizzerias_"+str(i)+".htm"))
        phone.append(page.find('section',attrs={'id':'details-card'}).find('p', attrs={'class':'phone'}).text)
        website.append("NA")
        add=str(page.find('section',attrs={'id':'details-card'}))
        idx1= add.find('Address: </span>')
        idx2=add.find('</p><p class')
        address.append(add[idx1+16:idx2])
    
print(address)
#print(phone)
#print(website)

['659 Valencia St, San Francisco, CA 94110', '5006 Mission St, San Francisco, CA 94112', '1042 Kearny St, San Francisco, CA 94133', '2800 Leavenworth St, Ste 50, San Francisco, CA 94133</p></section', '3881 24th St, San Francisco, CA 94114', '1570 Stockton St, San Francisco, CA 94133', '822 Irving St, San Francisco, CA 94122', '211 23rd Ave, San Francisco, CA 94121', '1737 Haight St, San Francisco, CA 94117', '420 Saint Francis Blvd, San Francisco, CA 94127</p></section', '3911 Alemany Blvd, San Francisco, CA 94132', '641 Vallejo St, San Francisco, CA 94133', '955 Geneva Ave, San Francisco, CA 94112', '2826 Jones St, San Francisco, CA 94133', '1052 Folsom St, San Francisco, CA 94103', '1980 Union St, San Francisco, CA 94123', '69 Cambon Dr, San Francisco, CA 94132', '279 Columbus Ave, San Francisco, CA 94133', '826 Folsom St, San Francisco, CA 94107', '1331 9th Ave, San Francisco, CA 94122', '2660 San Bruno Ave, San Francisco, CA 94134', '609 Vallejo St, San Francisco, CA 94133', '4077

In [740]:
len(website)

30

#### (9)  Sign up for a free account with https://positionstack.com/Links to an external site.  Copy your code from (8).  Modify the code to query each shop address’ geolocation (long, lat).  Update each shop document on the MongoDB collection “sf_pizzerias” to contain the shop’s address, phone number, website, and geolocation.

In [782]:
conn = http.client.HTTPConnection('api.positionstack.com')

In [781]:
lat_long_data =[]
for i in address:
    params = urllib.parse.urlencode({
    'access_key': 'd2ad8c8f81abf74631f213d89413309e',
    'query': i,
    'region': i,
    'limit': 1,
    })
    conn.request('GET', '/v1/forward?{}'.format(params))
    res = conn.getresponse()
    lat_long_data.append(res.read())

RemoteDisconnected: Remote end closed connection without response

In [780]:
#lat_long_data

In [776]:
#print(lat_long_data)
test=json.loads(lat_long_data[0])
test['data'][0]['latitude']

37.762347

In [777]:
geo_lat=[]
geo_long=[]

for i in lat_long_data:
    test=json.loads(i)
    geo_lat.append(test['data'][0]['latitude'])
    geo_long.append(test['data'][0]['longitude'])
    

In [778]:
print(len(geo_lat))
print(len(geo_long))

30
30


In [718]:
mycol_pizzas.find_one({})

{'_id': ObjectId('63ff1b3f7c91621e8c4bba20'),
 'sr_name': "Little Joe's Pizza",
 'sr': "Little Joe's Pizza",
 'name': "Little Joe's Pizza",
 'url_links': 'https://www.yellowpages.com/san-francisco-ca/mip/little-joes-pizza-461713663?lid=1002063125113'}

In [743]:
for idx,i in enumerate(products):
    i.update({'address':address[idx], 'phone' : phone[idx], 'website': website[idx],\
              'latitude' : geo_lat[idx] , 'longitude' : geo_long[idx]})

In [750]:
products[28]

{'sr_name': '29. Firehouse Pizzeria',
 'sr': '29',
 'name': ' Firehouse Pizzeria',
 'url_links': 'https://www.yellowpages.com/san-francisco-ca/mip/firehouse-pizzeria-480175297',
 '_id': ObjectId('6401464b7c91621e8c4bba61'),
 'address': '6001 California st, san francisco, CA 94121</p></section',
 'phone': 'Phone:  (415) 431-8800',
 'website': 'NA',
 'latitude': 37.783677,
 'longitude': -122.481915}

In [759]:
#mycol_pizzas.delete_many({})

In [754]:
for i in (products):
    mycol_pizzas.insert_one(i)

In [779]:
list(mycol_pizzas.find({}))

[{'_id': ObjectId('640171e76332be63ff92f3ca'),
  'sr_name': '1. Pizzeria',
  'sr': '1',
  'name': ' Pizzeria',
  'url_links': 'https://www.yellowpages.com/san-francisco-ca/mip/pizzeria-5920489',
  'address': '659 Valencia St, San Francisco, CA 94110',
  'geo_lat': 37.762347,
  'geo_long': -122.421423,
  'phone': 'Phone:  (415) 701-7492',
  'website': 'Website: https://www.facebook.com/yellowmotopizzeria'},
 {'_id': ObjectId('640171e76332be63ff92f3cb'),
  'sr_name': "2. Little Joe's Pizza",
  'sr': '2',
  'name': " Little Joe's Pizza",
  'url_links': 'https://www.yellowpages.com/san-francisco-ca/mip/little-joes-pizza-461713663?lid=1002063125113',
  'address': '5006 Mission St, San Francisco, CA 94112',
  'geo_lat': 37.718487,
  'geo_long': -122.439832,
  'phone': 'Phone:  (415) 333-3684',
  'website': 'Website: http://www.littlejoespizzeria.com'},
 {'_id': ObjectId('640171e76332be63ff92f3cc'),
  'sr_name': '3. Tommaso Ristorante Italiano',
  'sr': '3',
  'name': ' Tommaso Ristorante Ita